In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
%load_ext google.colab.data_table

Authenticated


In [ ]:
%%bigquery --project empik-ga360

SELECT
    PARSE_DATE('%Y%m%d', CAST(date AS STRING)) as true_date,
    concat(EXTRACT(ISOYEAR FROM PARSE_DATE('%Y%m%d', CAST(date AS STRING)) ), 
    EXTRACT(ISOWEEK FROM PARSE_DATE('%Y%m%d', CAST(date AS STRING)))) iso_week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    case when device.deviceCategory like 'mobile' then 'mobile' else 'desktop' end device, 
    trafficSource.source, trafficSource.medium, trafficSource.campaign, 
    sum(totals.visits) sesje, 
    sum(totals.totalTransactionRevenue/1000000) przychod, 
    sum(totals.transactions) transakcje
FROM
    `empik-ga360.304756.ga_sessions_*`
WHERE
    -- _table_suffix between '20211226' and '20211226'
    _table_suffix = FORMAT_DATE("%Y%m%d", DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
group by 1,2,3,4,5,6,7
order by true_date desc
limit 5

In [ ]:
%%bigquery --project empik-ga360

SELECT
    PARSE_DATE('%Y%m%d', CAST(date AS STRING)) as true_date,
    concat(EXTRACT(ISOYEAR FROM PARSE_DATE('%Y%m%d', CAST(date AS STRING)) ), 
    EXTRACT(ISOWEEK FROM PARSE_DATE('%Y%m%d', CAST(date AS STRING)))) iso_week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    sum(totals.visits) sesje, 
    sum(totals.totalTransactionRevenue/1000000) przychod, 
    sum(totals.transactions) transakcje
FROM
    `empik-ga360.304756.ga_sessions_*`
WHERE
    -- _table_suffix between '20211226' and '20211226'
    _table_suffix = FORMAT_DATE("%Y%m%d", DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
group by 1,2,3
order by true_date desc

users from GA

In [ ]:
%%bigquery --project empik-ga360

select 
u.true_date, 
u.iso_week_of_year, 
u.month_of_year, 
u.device, 
u.source,
u.medium, 
u.campaign, 
sum(u.uzytkownicy) users, 
sum(nu.uzytkownicy) 
new_usewrs 

from (
SELECT
    PARSE_DATE('%Y%m%d', CAST(date AS STRING)) as true_date,
    concat(EXTRACT(ISOYEAR FROM PARSE_DATE('%Y%m%d', CAST(date AS STRING))), 
    EXTRACT(ISOWEEK FROM PARSE_DATE('%Y%m%d', CAST(date AS STRING)))) iso_week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    case when device.deviceCategory like 'mobile' then 'mobile' else 'desktop' end device, 
    trafficSource.source, 
    trafficSource.medium, 
    trafficSource.campaign, 
    count(distinct fullVisitorId) uzytkownicy

FROM
    `empik-ga360.304756.ga_sessions_*`
WHERE
    -- _table_suffix between '20211226' and '20211226'
    _table_suffix = FORMAT_DATE("%Y%m%d", DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
group by 1,2,3,4,5,6,7
) u 

left join (
SELECT
    PARSE_DATE('%Y%m%d', CAST(date AS STRING)) as true_date,
    concat(EXTRACT(ISOYEAR FROM PARSE_DATE('%Y%m%d', CAST(date AS STRING))), 
    EXTRACT(ISOWEEK FROM PARSE_DATE('%Y%m%d', CAST(date AS STRING)))) iso_week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    case when device.deviceCategory like 'mobile' then 'mobile' else 'desktop' end device, 
    trafficSource.source, 
    trafficSource.medium, 
    trafficSource.campaign, 
    count(distinct fullVisitorId) uzytkownicy

FROM
    `empik-ga360.304756.ga_sessions_*`
WHERE
    -- _table_suffix between '20211226' and '20211226'
    _table_suffix = FORMAT_DATE("%Y%m%d", DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
    and visitNumber=1
group by 1,2,3,4,5,6,7
) nu on u.true_date=nu.true_date and u.device=nu.device and u.source=nu.source and u.medium=nu.medium and u.campaign=nu.campaign
group by 1,2,3,4,5,6,7
limit 5

GA Basic Metrics + Devices + Quantity

In [ ]:
%%bigquery --project empik-ga360

select 
b.true_date, 
b.iso_week_of_year, 
b.month_of_year, 
b.device,
b.source, 
b.medium, 
b.campaign, 
b.sesje, 
b.przychod, 
b.transakcje,
q.quantity  

from 
(SELECT
    PARSE_DATE('%Y%m%d', CAST(date AS STRING)) as true_date,
    concat(EXTRACT(ISOYEAR FROM PARSE_DATE('%Y%m%d', CAST(date AS STRING))), 
    EXTRACT(ISOWEEK FROM PARSE_DATE('%Y%m%d', CAST(date AS STRING)))) iso_week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    case when device.deviceCategory like 'mobile' then 'mobile' else 'desktop' end device, 
    trafficSource.source, 
    trafficSource.medium, 
    trafficSource.campaign, 
    sum(totals.visits) sesje, 
    sum(totals.totalTransactionRevenue/1000000) przychod, 
    sum(totals.transactions) transakcje
FROM
    `empik-ga360.304756.ga_sessions_*`
WHERE
    -- _table_suffix between '20211226' and '20211226'
    _table_suffix = FORMAT_DATE("%Y%m%d", DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
group by 1,2,3,4,5,6,7) b

left join (
SELECT
    PARSE_DATE('%Y%m%d', CAST(date AS STRING)) as true_date,
    concat(EXTRACT(ISOYEAR FROM PARSE_DATE('%Y%m%d', CAST(date AS STRING))), 
    EXTRACT(ISOWEEK FROM PARSE_DATE('%Y%m%d', CAST(date AS STRING)))) iso_week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    case when device.deviceCategory like 'mobile' then 'mobile' else 'desktop' end device, 
    trafficSource.source, trafficSource.medium, trafficSource.campaign, 
    sum(product.productQuantity) quantity
FROM
    `empik-ga360.304756.ga_sessions_*`, unnest(hits) as hits, unnest(hits.product) as product
WHERE
    -- _table_suffix between '20211226' and '20211226'
    _table_suffix = FORMAT_DATE("%Y%m%d", DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
    and eCommerceAction.action_type = '6' # Zrealizowany zakup = 6.
group by 1,2,3,4,5,6,7 
) q on b.true_date=q.true_date and b.device=q.device and b.source=q.source and b.medium=q.medium and b.campaign=q.campaign
limit 5